# AI 221: Machine Exercise 4 - Handwritten Digits Analysis

This exercise focuses on dimensionality reduction and classification techniques applied to the 8x8 handwritten digits dataset from sklearn.

# Dataset Preparation  

This section contains the loading the handwritten digits

# Exercises

This section contains the core implementations of this 

## Part A: Dimensionality Reduction Visualization [30 pts]

### Tasks:
1. Load and preprocess data:
   - Import digits dataset from sklearn.datasets
   - Apply StandardScaler normalization

2. Implement 6 dimensionality reduction techniques:
   - Local Linear Embedding (n_neighbors=200, random_state=0)
   - t-SNE (perplexity=50, random_state=0)
   - Isomap (n_neighbors=200)
   - Laplacian Eigenmap (n_neighbors=200)
   - Kernel PCA (kernel='rbf', gamma=0.01)
   - PCA

3. Visualization requirements:
   - Project data to 2D for each method
   - Color points by digit labels
   - Compare clustering quality

### Local Linear Embedding (n_neigbors = 200, random_state=0)

### t-SNE (perplexity = 50, random_state = 0)

### Isomap (n_neighbors=200)

### Laplacian Eigenmap (n_neighbors=200)


### Kernel PCA (kernel='rbf', gamma=0.01)

### PCA

### Results

<mark style="background-color: #fff000;"> Which of the methods produced clear clusters of data points? </mark>

## Part B: Variance Analysis [10 pts]

### Task Breakdown:
1. Generate CPV (Cumulative Percent Variance) plots for:
   - Kernel PCA
   - Standard PCA

2. Analysis requirements:
   - Determine number of components for 95% variance retention
   - Compare results between KPCA and PCA

### Kernel PCA

### Standard PCA

### Results


This section contains relevant inf

## Part C: Classification Comparison [60 pts]

### Data Preparation:
- Split data: 70% training, 30% testing
- Ensure stratified sampling by class label

### Classification Methods:

1. Method 1: Kernel PCA Pipeline [20 pts]
   - StandardScaler
   - Kernel PCA (kernel='sigmoid', n_components=40)
   - SVC (default parameters)
   - Report accuracy and F1-score

2. Method 2: LDA Pipeline [20 pts]
   - StandardScaler
   - LDA (n_components=9)
   - SVC (default parameters)
   - Report accuracy and F1-score
   - Explain LDA component limitation

3. Method 3: Baseline SVC [20 pts]
   - StandardScaler
   - SVC (default parameters)
   - No dimensionality reduction
   - Report accuracy and F1-score

### Analysis Requirements:
- Compare performance metrics
- Explain best performing method
- Discuss trade-offs

## Submission Guidelines

1. Format Requirements:
   - Submit as PDF file
   - Export from Jupyter Notebook
   - Highlight final answers clearly

2. Code Organization:
   - Clear section headers
   - Well-documented implementations
   - Proper visualization labels

3. Analysis Documentation:
   - Detailed explanations for each part
   - Comparative analysis between methods
   - Clear conclusions and insights

4. Submit through UVLE platform